In [ ]:
from __future__ import annotations

%load_ext jupyter_black

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt
import mesoscaler as ms

In [ ]:
_local_data = os.path.abspath("../data")

urma_store = os.path.join(_local_data, "urma.zarr")
assert os.path.exists(urma_store)
era5_store = os.path.join(_local_data, "era5.zarr")
assert os.path.exists(era5_store)

In [ ]:
from mesoscaler.enums import (
    # - ERA5
    GEOPOTENTIAL,
    SPECIFIC_HUMIDITY,
    TEMPERATURE,
    U_COMPONENT_OF_WIND,
    V_COMPONENT_OF_WIND,
    # - URMA
    SURFACE_PRESSURE,
    TEMPERATURE_2M,
    SPECIFIC_HUMIDITY_2M,
    U_WIND_COMPONENT_10M,
    V_WIND_COMPONENT_10M,
    SURFACE_PRESSURE,
)

era5_dvars = [
    GEOPOTENTIAL,
    TEMPERATURE,
    SPECIFIC_HUMIDITY,
    U_COMPONENT_OF_WIND,
    V_COMPONENT_OF_WIND,
]

urma_dvars = [
    SURFACE_PRESSURE,
    TEMPERATURE_2M,
    SPECIFIC_HUMIDITY_2M,
    U_WIND_COMPONENT_10M,
    V_WIND_COMPONENT_10M,
]

dataset_sequence = ms.open_datasets([(urma_store, urma_dvars), (era5_store, era5_dvars)])
dataset_sequence

In [ ]:
ratio = 2 / 1
height = 40  # px
dy = 100  # km
dx = int(dy * ratio)  # km
width = int(height * ratio)  # px
levels = [1013.25, 1000, 925, 850, 700, 600, 500, 400, 300, 200]
scale = ms.Mesoscale(dx, dy, levels=levels)
scale.plot.table()
plt.show()

In [ ]:
resampler = scale.resample(dataset_sequence, height=height, width=width)
resampler

In [ ]:
# - New Orleans
lon, lat, time = -90.0715, 29.9511, np.array(["2019-01-01T00:00:00.000000000"]).astype("datetime64[ns]")
batch = resampler(lon, lat, time)
print(batch.shape)

In [ ]:
from mesoscaler.sampling.resampler import PlotOption
from cartopy.feature import STATES, OCEAN

plotter = resampler.plot(lon, lat, time, features=[STATES, OCEAN])
plotter.shape

In [ ]:
options = [
    PlotOption("pressure", "contour", dim=0, colors="k", linestyles="-"),
    PlotOption("temperature", "contour", dim=1, colors="r", linestyles="--", linewidths=0.5),
    PlotOption("dp", "contourf", dim=2, cmap="Greens"),
    PlotOption("winds", "barbs", dim=(3, 4), alpha=0.5),
]
plotter.all_levels(0, options=options)

In [ ]:
# - mit campus
idx = (-71.0942, 42.3601, np.array(["2019-01-01T00:00:00.000000000"]).astype("datetime64[ns]"))


plotter(*idx).all_levels(0, options=options)

In [ ]:
# - scott-afb
idx = (-89.8446, 38.5454, np.array(["2019-01-01T00:00:00.000000000"]).astype("datetime64[ns]"))
plotter(*idx).all_levels(0, options=options)